In [159]:
try:
    import cirq
    import cirq_google
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq-google~=1.0.dev
    print("installed cirq.")
    import cirq
    import cirq_google


In [160]:
import numpy as np

Parity determining algorithm

In [161]:
inital = '012'
final = '201'

In [162]:
def generalized_hadamard(d):
    """
    Constructs the generalized Hadamard matrix for dimension d.

    Parameters:
        d (int): Dimension of the system (e.g., 2 for qubit, 3 for qutrit).

    Returns:
        numpy.ndarray: The d x d generalized Hadamard matrix.
    """
    # Compute the root of unity: e^(2πi/d)
    omega = np.exp(2j * np.pi / d)

    # Construct the Hadamard matrix
    hadamard_matrix = np.array([
        [omega, 1, omega**-1],
        [1, 1, 1],
        [omega**-1, 1, omega]
    ])

    # Normalize by 1/sqrt(d)
    return hadamard_matrix / np.sqrt(d)

In [163]:
def string_to_permutation_matrix(perm_string):
    """
    Converts a permutation string (e.g., "120") into a permutation matrix.

    Parameters:
        perm_string (str): A string representing the permutation.
                          Each character represents the index of the output basis.
                          For example, "120" means |0⟩ -> |1⟩, |1⟩ -> |2⟩, |2⟩ -> |0⟩.

    Returns:
        numpy.ndarray: The permutation matrix as a 2D NumPy array.
    """
    # Length of the permutation
    n = len(perm_string)

    # Initialize an n x n zero matrix
    permutation_matrix = np.zeros((n, n), dtype=int)

    # Fill the permutation matrix based on the string
    for i, char in enumerate(perm_string):
        output_index = int(char)  # Get the integer index from the string
        permutation_matrix[output_index, i] = 1  # Place a 1 at the correct position

    return permutation_matrix

In [164]:
#Creating hadamad and permutation gates
H3_gate = cirq.MatrixGate(generalized_hadamard(3), qid_shape=(3,))

# Define the permutation matrix
permutation_matrix = string_to_permutation_matrix(final)

# Create the permutation gate
permutation_gate = cirq.MatrixGate(permutation_matrix, qid_shape=(3,))

# Compute the inverse Hadamard matrix
inverse_hadamard_matrix = np.conjugate(generalized_hadamard(3).T)
# Create the inverse Hadamard gate
inverse_hadamard_gate = cirq.MatrixGate(inverse_hadamard_matrix, qid_shape=(3,))

In [165]:
def printstate(state_vector,Tndex):
    # Print amplitudes with their index positions
    print(f"State = ∣{Tndex}>")
    for index, amplitude in enumerate(state_vector):
        print(f"State {Tndex[index]}: Amplitude {amplitude}, Probability {np.abs(amplitude)**2}")

In [166]:
#creating a qutrit
qutrit = cirq.LineQid(0, dimension=3)
# Get the final state vector from the simulation and Print amplitudes with their index positions
simulator = cirq.Simulator()

#applying the 3state hadamad gate to the qutrit
circuit = cirq.Circuit(H3_gate(qutrit))
printstate(simulator.simulate(circuit).final_state_vector,inital)
#print cirucuit
print(circuit)

#applying the permutation matrix
circuit.append(permutation_gate.on(qutrit))
# Get the final state vector from the simulation and Print amplitudes with their index positions
printstate(simulator.simulate(circuit).final_state_vector,final)
print(circuit)

#applying qft which is just same as hadamard so using that again
circuit.append(inverse_hadamard_gate.on(qutrit))
# Get the final state vector from the simulation and Print amplitudes with their index positions
printstate(simulator.simulate(circuit).final_state_vector,inital)
print(circuit)

State = ∣012>
State 0: Amplitude (-0.28867512941360474+0.5j), Probability 0.33333332136784577
State 1: Amplitude (0.5773502588272095+0j), Probability 0.33333332136784577
State 2: Amplitude (-0.28867512941360474-0.5j), Probability 0.33333332136784577
            ┌                                   ┐
            │-0.289+0.5j  0.577+0.j  -0.289-0.5j│
0 (d=3): ───│ 0.577+0.j   0.577+0.j   0.577+0.j │───
            │-0.289-0.5j  0.577+0.j  -0.289+0.5j│
            └                                   ┘
State = ∣201>
State 2: Amplitude (0.5773502588272095+0j), Probability 0.33333332136784577
State 0: Amplitude (-0.28867512941360474-0.5j), Probability 0.33333332136784577
State 1: Amplitude (-0.28867512941360474+0.5j), Probability 0.33333332136784577
            ┌                                   ┐   ┌     ┐
            │-0.289+0.5j  0.577+0.j  -0.289-0.5j│   │0 1 0│
0 (d=3): ───│ 0.577+0.j   0.577+0.j   0.577+0.j │───│0 0 1│───
            │-0.289-0.5j  0.577+0.j  -0.289+0.5j│   │1 0 0│
    